In [30]:
# import libraries in dictionary style
import pandas as pd
import datetime
import numpy as np
import MySQLdb

libraries = {'numpy': np, 'pandas': pd, 'datetime': datetime}

In [31]:
# database credentials
dsn_database = "ercm_kc135"
dsn_hostname = "localhost"
dsn_port = 3306
dsn_uid = "root"
dsn_pwd = "root"

# create the database connection
conn = MySQLdb.connect(host = dsn_hostname, port = dsn_port, user = dsn_uid, passwd = dsn_pwd, db = dsn_database)

In [32]:
# create an empty dataframe
df = []

# select last five years of sortie history
query = "SELECT * FROM ercm_kc135.compiled_sortie_history_data"

# create dataframe from sortie history
df = pd.read_sql_query(query, conn)

In [33]:
# set the period (will make this a parameter eventually)
period = 3
date_selection = 'Last_Record'

In [34]:
def limit_data_set(df, dselect, libraries):
    """ Limits the data set to only necessary columns and only relevant dates
    Args:
        df: dateframe from previous function
        dselect: determines what will set the current date
        libraries: dictionary of libraries; access by name
        e.g. pd = libraries['pandas'] or stats = libraries['scipy']['stats']
    Returns:
        df: data frame with limited data
    """
    dt = libraries['datetime']

    # limit the data frame to desired columns and the last three years
    df = df[['Serial_Number', 'Depart_Date', 'Flying_Hours']]

    if dselect == 'Today':
        ldate = dt.datetime.now().year - 5
    else:
        ldate = df['Depart_Date'].dt.year.max() - 5

    # limit the data frame
    df = df[df['Depart_Date'].dt.year >= ldate]

    # calculate the first of the month
    df['Fixed_Date'] = pd.to_datetime(df['Depart_Date']).apply(lambda x: '{year}-{month}-01'.format(year=x.year, month=x.month) if x.month > 9 else '{year}-0{month}-01'.format(year=x.year, month=x.month))
    df['Fixed_Date'] = df['Fixed_Date'].map(lambda x: np.datetime64(x))
    

    return df, ldate

In [36]:
df, last_date = limit_data_set(df, date_selection, libraries)
# print df.head(200)
# print df.dtypes

In [37]:
def calc_monthly_values(df, libraries):
    """ Calculates the number of unique tail numbers present in each time window.
    Args:
        df: date frame from previous function
        libraries: dictionary of libraries; access by name
    Returns:
        tailcount: count of unique tail_numbers in each window
    """
    pd = libraries['pandas']
    
    # subset values to only be necessary columns
    df = df[['Serial_Number','Flying_Hours', 'Fixed_Date']]

    #group by month and count distinct tails
    totals = df.groupby(['Fixed_Date'], as_index=False).agg({'Serial_Number': pd.Series.nunique, 'Flying_Hours': pd.Series.sum}) 
    
    # rename the columns
    totals.rename(columns={'Serial_Number': 'Unique_Tail_Count', 'Flying_Hours': 'Total_Flying_Hours'}, inplace=True)
    
    # calculate the monthly average
    totals['Monthly_Average'] = totals['Total_Flying_Hours'] / totals['Unique_Tail_Count']
    
    # convert to data frame
    totals = pd.DataFrame(totals)
    
    # drop intermediate columns
    totals.drop(['Unique_Tail_Count', 'Total_Flying_Hours'], axis=1, inplace = True)
    
    return totals

In [38]:
totals = calc_monthly_values(df, libraries)
# print totals.head()
# print totals.dtypes

  Fixed_Date  Monthly_Average
0 2013-01-01        36.813684
1 2013-02-01        36.553425
2 2013-03-01        35.246900
3 2013-04-01        36.233947
4 2013-05-01        37.288283
Fixed_Date         datetime64[ns]
Monthly_Average           float64
dtype: object


In [160]:
def calc_windows(df, p, libraries):
    """ Calculates window value columns to determine what periods a flight record could be grouped into.
    Example: '2016-01-04' would return window values '2015-11-01', '2015-12-01', '2016-01-01'.
    Args:
        df: date frame from previous function
        p: currently only works where period = 3. Long-term, will try and make dynamic based on this value
        libraries: dictionary of libraries; access by name
        e.g. pd = libraries['pandas'] or stats = libraries['scipy']['stats']
    Returns:
        windowed: dataframe with window values where record is active
    """
    np = libraries['numpy']

    # create useful window columns by iterating for 1 through number of periods specified
    for i in range(1, p + 1):
        # create column name and set window value (either 2, 1, or 0 when period = 3)
        column = 'W' + str(i)
        window = p - i

        # extract baseline month values
        df['Month'] = df['Fixed_Date'].dt.month
        df['Year'] = df['Fixed_Date'].dt.year

        # subset with loc to fix problematic time periods with year overlaps
        # if window is 2 and month is Jan or Feb, fix date accordingly
        if window == 2:
            df.loc[(df.Month == 1) | (df.Month == 2), 'Year'] = df.Year - 1
            df.loc[(df.Month == 2), 'Month'] = 14
            df.loc[(df.Month == 1), 'Month'] = 13
        # if window is 1 and month is Jan, fix date accordingly
        elif window == 1:
            df.loc[(df.Month == 1), 'Year'] = df.Year - 1
            df.loc[(df.Month == 1), 'Month'] = 13

        # Decrement them month value based on the window value
        df['Month'] = df.Month - window

        # clean up date integer values by adding appropriate 0's and converting to strings
        df.loc[df.Month < 10, 'Month'] = '0' + df['Month'].astype(str)  # add 0's to single digit months
        df.loc[df.Month >= 10, 'Month'] = df['Month'].astype(str)  # do not add 0's to double digit months
        df['Year'] = df['Year'].astype(str)

        # create the value for each window as a string
        df[column] = df['Year'] + '-' + df['Month'] + '-01'

        # convert each window back to original datetime64 format
        df[column] = df[column].map(lambda x: np.datetime64(x))

    # drop the month and year columns
    windowed = df.drop(['Month', 'Year'], axis=1)

    return windowed

In [161]:
def calc_rolling_avg(df, last_date, p, libraries):
    """ Calculates the rolling average by melting the window columns and grouping the result.
    Args:
     df: date frame from previous function
     tailcount: count of unique tails active in each month
     last_date: year associated with five years ago
     p: currently only works where period = 3. Long-term, will try and make dynamic based on this value
     libraries: dictionary of libraries; access by name
        e.g. pd = libraries['pandas'] or stats = libraries['scipy']['stats']
    Returns:
    grouped: data frame with the moving average of flying_hours
    """
    pd = libraries['pandas']

    # create a list of column values thus far
    columns = list(df.columns.values)

    # calculate the point at which the melted data frame with pivot
    melt_axis = len(columns) - p

    # create a melted data frame to used for group by calculation
    melted = pd.melt(df,
                     id_vars=columns[1:melt_axis],
                     value_vars=columns[melt_axis:len(columns)],
                     var_name='Window',
                     value_name='Window_Val'
                     )

    # group by window value and calculate the mean
    grouped = melted.groupby(['Window_Val'], as_index=False).sum()

    # rename columns
    grouped.rename(columns={'Window_Val': 'Three_Month_Start_Date', 'Monthly_Average': 'Average_Flying_Hours'}, inplace=True)

    # eliminate windows that predate the start year
    grouped = pd.DataFrame(grouped[grouped['Three_Month_Start_Date'].dt.year >= last_date])
    

    
    # eliminate the last two windows
    grouped.drop(grouped.tail(2).index, inplace=True)

    return melted, grouped

In [162]:
windowed = calc_windows(totals, period, libraries)
melted, final = calc_rolling_avg(windowed, last_date, period, libraries)

# print totals.head()
# print windowed.head()
# print melted.head()
print final.head(150)

   Three_Month_Start_Date  Average_Flying_Hours
2              2013-01-01            108.614009
3              2013-02-01            108.034272
4              2013-03-01            108.769131
5              2013-04-01            110.208916
6              2013-05-01            112.175247
7              2013-06-01            116.549394
8              2013-07-01            117.763543
9              2013-08-01            115.581446
10             2013-09-01            109.736812
11             2013-10-01            106.750847
12             2013-11-01            104.584654
13             2013-12-01            106.754788
14             2014-01-01            116.643897
15             2014-02-01            125.995451
16             2014-03-01            131.443654
17             2014-04-01            131.341376
18             2014-05-01            131.698112
19             2014-06-01            134.636000
20             2014-07-01            135.762591
21             2014-08-01            143